## More data Processing with Pandas
- Reading assignments
    - Chapter eight: Data Wrangling: Join, Combine, Reshape
    - Chapter ten: Data Aggregation and group operations
    - Chapter twelve: Advanced pandas
- Merging DataFrames
- Pandas Idiom
- Group by
- Scales
- Pivot Table
- Date/Time functionality

In [2]:
# Aqui estaremos usando las teorias de conjuntos que se conocen de los Diagramas de Venn.
# Este background se traducira utilizando Pandas. Hay que reconocer las distitnas convenciones 
# como outer-full-join, inner-join, etc y su relacion con las uniones e intercepciones

import pandas as pd
# Primero creamos dos DataFrames, staff y students
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                          {'Name': 'Sally', 'Role': 'Course liasion'},
                          {'Name': 'James', 'Role': 'Grader'}])
# indexamos por nombre
staff_df = staff_df.set_index('Name')

# Creamos un DataFrame para los estudiantes
student_df = pd.DataFrame([{'Name':'James', 'School':'Business'},
                            {'Name':'Mike', 'School':'Law'},
                           {'Name': 'Sally','School': 'Engineering'}])
# Indexamos por nombre
student_df = student_df.set_index('Name')



# Imprimimos los df
print(staff_df)
print(student_df)



                 Role
Name                 
Kelly  Director of HR
Sally  Course liasion
James          Grader
            School
Name              
James     Business
Mike           Law
Sally  Engineering


In [3]:
# Aqui lo importante es notar que tenemos a Sally y James en ambos sets. Son estudiantes y Staff pero
# Mike y Kelly No.

In [4]:
# Si queremos hacer una union, llamamos merge() pasando el DataFrame en la izquierda y el DataFrame en la derecha.
# Le decimos a merge() que queremos un outer join y que queremos que use los indices de la izquierda y la derecha
# como las joining columns

pd.merge(staff_df,student_df,how='outer', left_index = True, right_index = True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [5]:
# Aqui se hizo una union de todo. Ambos conjuntos se encuentra. Vemos que Mike no tiene un Role, y Kelly no tiene
# una School y se listan estas celdas como missing values

# Si queremos una interseccion, solamente los estudiantes Y staff, usamos el atributo inner. 
# De nuevo, seleccionamos el indice de la izquierda y de la derecha como True para que sean las joining columns

pd.merge(staff_df,student_df, how= 'inner', left_index= True, right_index = True)


,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


In [6]:
# Podemos hacer set addition. Por ejemplo, queremos una lista de todo el staff, sin importar que sean estudiantes
# o no. Pero, si son estudiantes, queremos que se muestren esos detalles. Para esto vemos que merge respeta
# el orden en el que le pasamos los DataFrames. En este caso, usamos left join

pd.merge(staff_df,student_df,how='left',left_index=True,right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


In [7]:
# Este es nuestro Staff. Pero vemos que Sally y James, que son tambien estudiantes, muestran su escuela

# Hagamos esto para el conjunto de estudiantes. Esta vez usamos un right join y nos mostrata los estudiantes
# Sin importar que sean staff. Y cuando son staff, nos mostrara los detalles

pd.merge(staff_df,student_df, how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


In [8]:
# Hay otro parametros de merge que son interesantes. Por ejemplo, no necesitas hacer el join con los indices,
# puedes hacerlo usando las columnas. Puedes usar join on. 

# Por ejemplo, queremos usar on y una columna que este en ambos DataFrames para hacer el joining

#Removeremos los indices de ambos DataFrames
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

# Ahora hacemos merge usando el parametro on
pd.merge(staff_df,student_df, how = 'right', on = 'Name')

,Name,Role,School
0,Sally,Course liasion,Engineering
1,James,Grader,Business
2,Mike,NaN,Law


In [9]:
# Usando el parametro "on" en lugar del indice es la forma más comun de encontrar los merges

In [11]:
# Que ocurre cuando hay conflicto entre DataFrames?. 

staff_df= pd.DataFrame([{'Name':'Kelly', 'Role':'Director of HR', 'Location':'State Street'},
                        {'Name':'Sally', 'Role':'Course liasion','Location':'Washington Avenue'},
                        {'Name':'James', 'Role':'Grader','Location': 'Washington Avenue'}])

student_df = pd.DataFrame([{'Name':'James','School':'Business','Location':'1024 Billiard Avenue'},
                           {'Name':'Mike','School':'Law','Location':'Fraternity House #22'},
                           {'Name':'Sally','School':'Engineering','Location':'512 Wilson Crescent'}])

# En el staff DataFrame tenemos la direccion de la oficina donde podemos encontrar al staff. En el
# DataFrame de estudiantes tenemos su direccion personal. 

# La funcion merge preserva esta informacion pero añade un _x or _y para ayudar a diferenciar entre
# cual indice con cual columna de datos. El _x siempre se refiere a la informacion del DataFrame de la izquierda
# mientras que el _y siempre se refiere a la informacion del DataFrame de la derecha

# Si queremos la informacion del staff, sin importar si son estudiantes o no, pero, si son estudiantes nos gustaria
# tener sus detalles, podemos usar el left join en la columna de nombre

pd.merge(staff_df,student_df, how= 'left', on='Name')


,Location_x,Name,Role,Location_y,School
0,State Street,Kelly,Director of HR,NaN,NaN
1,Washington Avenue,Sally,Course liasion,512 Wilson Crescent,Engineering
2,Washington Avenue,James,Grader,1024 Billiard Avenue,Business


In [12]:
# Aqui vemos las columas Location_x y Location_y. La columna Location_x se refiere a la informacion del 
# left DataFrame, en este caso, contiene la direccion del staff.

# Que pasa cuando tenmos overlaping? Por ejemplo, un DataFrame que tenga el nombre y el apellido donde hayan
# personas con el mismo nombre pero distinto apellido? Si hacemos on Name va a haber overlaping 
# En este caso, usamos una lista de multiples columnas que deberian ser usadas como join keys de ambos DataFrames


staff_df= pd.DataFrame([{'First Name':'Kelly', 'Last Name': 'Desjardins', 'Role':'Director of HR'},
                        {'First Name':'Sally', 'Last Name':'Brooks','Role':'Course liasion'},
                        {'First Name':'James', 'Last Name':'Wilde', 'Role':'Grader'}])

student_df = pd.DataFrame([{'First Name':'James', 'Last Name':'Hammond','School':'Business'},
                           {'First Name':'Mike','Last Name':'Smith','School':'Law'},
                           {'First Name':'Sally','Last Name':'Brooks','School':'Engineering'}])

# James Wilde y James Hammond no coinciden en ambas keys debido a que tienen distintos apellidos
# Por lo que podriamos esperar que un inner join no incluya a ambos. Pero a Sally si

pd.merge(staff_df,student_df, how= 'inner', on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering
